In [ ]:
#Load libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from random import randrange, uniform

In [ ]:
#Set working directory
os.chdir("D:\Data Science\Assignments\Project")

In [ ]:
#Load data
train = pd.read_csv("train.csv")

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
test.info()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
test.describe()

# Missing Value Analysis

In [ ]:
#missing value 
missing_val = train.isnull().sum()

In [ ]:
missing_val

# Outlier Analysis

In [ ]:
%matplotlib inline
import seaborn as sns

In [ ]:
# to see the distribution of target variable
sns.countplot(x= 'target', data= train)

In [ ]:
cnames = train.drop(['target', 'ID_code'], axis= 1)

In [ ]:
cnames.columns

In [ ]:
#Detect and delete outliers from data
for i in cnames:
    print (i)
   
    q75, q25 = np.percentile(train.loc[:,i], [75 ,25])
    iqr = q75 - q25

    minimum = q25 - (iqr*1.5)
    maximum = q75 + (iqr*1.5)
    print (minimum)
    print (maximum)
   
    
    train = train.drop(train[train.loc[:,i] < minimum].index)
    train = train.drop(train[train.loc[:,i] > maximum].index)

In [ ]:
train.shape            

In [ ]:
train['target'].value_counts()

In [ ]:
train['target'].value_counts()*100/len(train)

In [ ]:
train['target'].hist(bins=5)

# Feature selection

In [ ]:
#correlation matrix

corr=train.iloc[:,:25].corr()

corr1=train.iloc[:,174:].corr()

In [ ]:
#Set the width and hieght of the p
fig, ax=plt.subplots(figsize=(20,20))

# plot the heatmap

sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(130, 10, as_cmap=True),
            square=True, ax=ax, annot =True, vmin =1, linewidth = 0.2)

# Feature scaling

In [ ]:
#import sklearn

from sklearn import preprocessing

In [ ]:
#let's check distribution of the data for first 25 columns

print('Distributions of first 25 columns')
plt.figure(figsize=(26, 24))
for i, col in enumerate(list(train.columns)[2:27]):
    plt.subplot(7, 4, i + 1)
    plt.hist(train[col])
    plt.title(col)

In [ ]:
#Nomalisation
for i in cnames:
    print(i)
    train[i] = (train[i] - np.min(train[i]))/(np.max(train[i]) - np.min(train[i]))

In [ ]:
train.head()

# Model Development

In [ ]:
#import libraries

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
minority_class_len = len(train[train['target']== 1])

print(minority_class_len)

In [ ]:
majority_class_indices = train[train['target']==0].index
print(len(majority_class_indices))
print(majority_class_indices)

In [ ]:
random_majority_indices = np.random.choice(majority_class_indices, minority_class_len, replace = False)

print(random_majority_indices)

In [ ]:
minority_class_indices = train[train['target']==1].index
print(minority_class_indices)

In [ ]:
under_sample_indices = np.concatenate([minority_class_indices, random_majority_indices])

In [ ]:
train = train.loc[under_sample_indices]

In [ ]:
sns.countplot(x= 'target', data= train)

In [ ]:
#replace target variables values with yes or no

train['target'] = train['target'].replace(0, 'No')
train['target'] = train['target'].replace(1, 'Yes')

In [ ]:
train.head()

In [ ]:
#divide the dataset in test and train data

X = train.drop(['ID_code', 'target'], axis=1)
Y = train['target']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
#decision tree

from sklearn import tree

C50_model = tree.DecisionTreeClassifier(criterion='entropy').fit(X_train, y_train)

#test scores

C50_predictions = C50_model.predict(X_test)

In [ ]:
#confusion matrix

Confusion_matrix = CM = pd.crosstab(y_test, C50_predictions)

#let check for belw values
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

In [ ]:
Confusion_matrix

In [ ]:
#Accurary for the model decision tree
Accuracy = ((TP+TN)*100)/(TP+TN+FP+FN)

In [ ]:
Accuracy

In [ ]:
#False Negative rate for the model decision tree
FNR = (FN*100)/(FN+TP)

In [ ]:
FNR

In [ ]:
#recall for the model decision tree 
print('Recall: ', TP/(TP+FN))

#precision for the model decision tree
print('Precision:', TP/(TP+FP) )

In [ ]:
# random forest

from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)

In [ ]:
Prediction = RF.predict(X_test)

In [ ]:
#confusion matrix
Confusion_matrix = CM = pd.crosstab(y_test, Prediction)

#lets check the below values
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]


In [ ]:
Confusion_matrix

In [ ]:
#Accurary for the model random forest(N=100)
print('Accuracy: ', ((TP+TN)*100)/(TP+TN+FP+FN))

#false negative rate for the model random forest(N=100)
print('FNR:', (FN*100)/(FN+TP))

In [ ]:
#recall for the model random forest(N=500)
print('Recall: ', TP/(TP+FN))

#precision for the model random forest(N=500)
print('Precision:', TP/(TP+FP))

FOR RANDOM FOREST MODEL
with n =500
Accuracy = 75.94
FNR = 20.55
Recall = 0.79
precision = 0.73

with n =100
Accuracy = 89.93
FNR = 99.93

In [ ]:
from sklearn import metrics

print('Accuracy:', metrics.accuracy_score(y_test, Prediction)*100)

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train)

In [ ]:
Predict = KNN.predict(X_test)

In [ ]:
Confusion_matrix = CM = pd.crosstab(y_test, Predict)

Confusion_matrix

In [ ]:
#let us save TP, TN, FP, FN

TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model

((TP+TN)*100)/(TP+TN+FP+FN)

In [ ]:
#False Negative rate 
(FN*100)/(FN+TP)

In [ ]:
#recall 
print('Recall: ', TP/(TP+FN))

#precision
print('Precision:', TP/(TP+FP) )

FOR K NEAREST NEIGHBOUR MODEL
Accuracy = 56.4
FNR = 83
Recall = 0.17
precision = 0.18

In [ ]:
# logistic regression

from sklearn.linear_model import LogisticRegression

#instantiate the model

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

In [ ]:
Pred = logreg.predict(X_test)

In [ ]:
CM = pd.crosstab(y_test, Pred)


#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy

((TP+TN)*100)/(TP+TN+FP+FN)


In [ ]:
#False negative

(FN*100)/(FN+TP)

In [ ]:
#recall 
print('Recall: ', TP/(TP+FN))

#precision
print('Precision:', TP/(TP+FP) )

In [ ]:
probs = logreg.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_aucc = metrics.auc(fpr, tpr)

In [ ]:
plt.title('AUC CURVE')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_aucc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

FOR LOGISTIC REGRESSION MODEL
Accuracy = 76.98
FNR = 23.03
Recall = 0.77
precision = 0.76

In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB

#implementation

Navbay = GaussianNB().fit(X_train, y_train)

In [ ]:
PredictNB = Navbay.predict(X_test)

In [ ]:
CM = pd.crosstab(y_test, PredictNB)


#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy

((TP+TN)*100)/(TP+TN+FP+FN)

In [ ]:
#False negative

(FN*100)/(FN+TP)

In [ ]:
#recall 
print('Recall: ', TP/(TP+FN))

#precision
print('Precision:', TP/(TP+FP) )

In [ ]:
y_true = train.iloc[:,1]
y_probas = test['Actual_prob']
skplt.metrics.plot_roc_curve(y_true, y_probas)
plt.show()

In [ ]:
probs = NB_model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
plt.title('AUC CURVE')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

FOR NAIVE BAYES MODEL
Accuracy = 80.15
FNR = 20.75
Recall = 0.79
precision = 0.80
AUC CURVE = 0.89

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(loss= 'modified_huber', shuffle= True, random_state= 101)

In [ ]:
sgd.fit(X_train, y_train)

In [ ]:
PredictSGD = sgd.predict(X_test)

In [ ]:
CM = pd.crosstab(y_test, PredictSGD)

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy

((TP+TN)*100)/(TP+TN+FP+FN)

In [ ]:
#False negative

(FN*100)/(FN+TP)

In [ ]:
#recall 
print('Recall: ', TP/(TP+FN))

#precision
print('Precision:', TP/(TP+FP) )

FOR SGDClssifier MODEL
Accuracy = 60.31
FNR = 1.94
Recall = 0.98
precision = 0.55